In [17]:
import pandas as pd
import warnings
import math
from matplotlib import pyplot as plt
from datetime import datetime
import sys
import numpy as np

In [20]:
print("Hora:", datetime.now().strftime("%H:%M:%S"))

Hora: 14:23:01


In [22]:
states = ['SE']
years = list(range(2014,2025))

# general path
general_path = 'C:/Users/abner/OneDrive/Documentos/mestrado/DISSERTAÇÃO/scripts'
mit_int=360
method = "RUSLE2"   # choose"USLE", "RUSLE" ou "RUSLE2"


pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', None)



#HQ LIST
#df_tel_hq=pd.read_excel(general_path+'/Estacoes_telemetria_HQ.xlsx')
df_cem_hq=pd.read_csv(general_path+'/Estacoes_Alta_Qualidade_da_Analise_Visual.csv',sep=';')

In [24]:
def calculate_KE(I, method):
    if method.upper() == "USLE":
        if I <= 76:
            return 0.119 + 0.0873 * math.log10(I)
        else:
            return 0.283
    elif method.upper() == "RUSLE":
            return 0.29 * (1 - 0.72 * math.exp(-0.05 * I))
    elif method.upper() == "RUSLE2":
        return 0.29 * (1 - 0.72 * math.exp(-0.082 * I))

In [16]:
for state in ['SC']:
    for year in [2024]:
        start_time = datetime.now()
        mit=pd.Timedelta(mit_int, unit ='m')
        name_file= state+ '_'+ str(year)+'.h5'
        name_col= state+ '_ '+ str(year)
        print(f"Initialize : {state}_{year} {start_time.strftime("%Y-%m-%d %H:%M:%S")}") 
        df_rainfall = pd.read_hdf(general_path +'/1 - Organized data/'+ name_file,'table_data')

        monthly_results = {m: {"gauge_code": [], "EI30": [], "E_D": [], "rain": []} for m in range(1, 13)}
        annual_results = {"gauge_code": [], "EI30": [], "E_D": [], "rain": []}
        for code in df_cem_hq[name_col].dropna():
            start_time_code = datetime.now()  
            name=code +'_' + str(mit_int)
            df_rainfall_events = pd.read_hdf(general_path + '/3 - Rainfall events/'+ name_file, name).drop_duplicates()
            df_partial_result = pd.DataFrame(columns=['rain_mm','duration', 'I30', 'RF_KE', 'EI30'])
            for evento in range(0,len(df_rainfall_events)):
                beg=df_rainfall_events.iloc[evento,1]
                month = beg.month
                end=df_rainfall_events.iloc[evento,1]+pd.Timedelta(hours=df_rainfall_events.iloc[evento,3])
                df_gauge_rainfall=df_rainfall[df_rainfall['gauge_code']==code]
                df_event = df_gauge_rainfall[(df_gauge_rainfall['datetime'] >= beg) & (df_gauge_rainfall['datetime'] <= end)].reset_index(drop=True)

                if df_event['rain_mm'].sum()>12.7:  #SPECIFICATION AT RUSLE 2
                    for i in df_event.index:
                        RF=df_event['rain_mm'][i] #mm
                        RF_intensity=(RF/10)*60 #mm/h  ###tdel
                        #kinectic energy
                        KE = calculate_KE(RF_intensity, method) if RF != 0 else 0
                        RF_KE=KE*RF#MJ/ha
                        if (KE > 0.283) :KE= 0.283 #max value 
                        
                        #calculate I30
                        tbeg=df_event.iloc[i,3] 
                        tend=df_event.iloc[i,3]+pd.Timedelta(hours=0.5)
                        df_temp=df_event[(df_event['datetime'] >= tbeg) & (df_event['datetime'] < tend)]#### < ou <=
                        I30=df_temp['rain_mm'].sum()/0.5 #mm/h #intervalo de 0,5 hrs

                        df_event.at[i,'Rf_intensity']=RF_intensity
                        df_event.at[i,'RF_KE']=RF_KE
                        df_event.at[i,'I30']=I30

                    #calc ei30       
                    EI30=df_event['RF_KE'].sum()*df_event['I30'].max()        
                    #set partial results
                    df_partial_result.at[evento,'rain_mm']=df_event['rain_mm'].sum() 
                    df_partial_result.at[evento,'duration']=df_rainfall_events.iloc[evento,3]        
                    df_partial_result.at[evento,'I30']=df_event['I30'].max() 
                    df_partial_result.at[evento,'RF_KE']=df_event['RF_KE'].sum()
                    df_partial_result.at[evento,'EI30']=EI30
                    df_partial_result.at[evento,'month']=month
                
                else:

                    df_partial_result.at[evento,'rain_mm']=df_rainfall_events.iloc[evento,2]  
                    df_partial_result.at[evento,'duration']=df_rainfall_events.iloc[evento,3]        
                    df_partial_result.at[evento,'I30']=0
                    df_partial_result.at[evento,'RF_KE']=0
                    df_partial_result.at[evento,'EI30']=0
                    df_partial_result.at[evento,'month']=month
                #set final results        
            with pd.HDFStore(general_path + f'/1 - Results_Cemaden/Results_{state}_{year}.h5', 'a') as store:
                for month in range(1, 13):
                #set monthly results
                    df_month = df_partial_result[df_partial_result['month']==month]
                    EI30_M=df_month['EI30'].sum()
                    rain=df_month['rain_mm'].sum()
                    E_D = EI30_M / rain if rain != 0 else float('nan')
                    monthly_results[month]["gauge_code"].append(code)
                    monthly_results[month]["EI30"].append(EI30_M)
                    monthly_results[month]["E_D"].append(E_D)
                    monthly_results[month]["rain"].append(rain)                
                    
                    store.put(f'month_{month:02d}',pd.DataFrame(monthly_results[month]))
            
                #set anual results
                
                EI30_T=df_partial_result['EI30'].sum()
                E_D=EI30_T/df_partial_result['rain_mm'].sum()
                rain=df_partial_result['rain_mm'].sum()
                annual_results["gauge_code"].append(code)
                annual_results["EI30"].append(EI30_T)
                annual_results["E_D"].append(E_D)
                annual_results["rain"].append(rain)
                store.put('annual', pd.DataFrame(annual_results))


            print(f"  -code: {code} , time: {(datetime.now()-start_time_code).total_seconds():.2f}s , {len(annual_results['E_D'])}/{len(df_cem_hq[name_col].dropna())}")

        minutes, seconds = divmod((datetime.now()-start_time).total_seconds(), 60)#total duration
        print(f"{state}_{year} completed, total duration: {int(minutes)}m {seconds:.2f}s")         

Initialize : SC_2024 2025-04-05 18:22:53
  -code: 420090301A , time: 29.39s , 1/58
  -code: 420120802A , time: 32.21s , 2/58
  -code: 420130701A , time: 26.20s , 3/58
  -code: 420130703A , time: 29.75s , 4/58
  -code: 420130704A , time: 27.83s , 5/58
  -code: 420140601A , time: 27.52s , 6/58
  -code: 420150501A , time: 25.91s , 7/58
  -code: 420180202A , time: 27.51s , 8/58
  -code: 420207301A , time: 23.98s , 9/58
  -code: 420230501A , time: 24.27s , 10/58
  -code: 420240401G , time: 25.90s , 11/58
  -code: 420240404A , time: 25.26s , 12/58
  -code: 420240406A , time: 23.93s , 13/58
  -code: 420240407G , time: 31.11s , 14/58
  -code: 420240408G , time: 33.25s , 15/58
  -code: 420240412G , time: 27.46s , 16/58
  -code: 420240413G , time: 26.48s , 17/58
  -code: 420240423A , time: 23.83s , 18/58
  -code: 420300601A , time: 23.41s , 19/58
  -code: 420419401A , time: 29.87s , 20/58
  -code: 420420201A , time: 19.66s , 21/58
  -code: 420430102A , time: 23.10s , 22/58
  -code: 420450903A , 

In [ ]:
with pd.HDFStore(general_path + f'/1 - Results_Cemaden/Results_{'AC_ 2019'}.h5') as hdf:
    keys = hdf.keys()  # Obtém todas as chaves do arquivo HDF
    for key in keys:  # Itera por cada chave
        df = pd.read_hdf(hdf, key=key)  # Lê os dados de cada chave
        print(f"Chave: {key}, Número de linhas: {len(df)}")


import h5py

# Abrir o arquivo HDF5 no modo leitura/escrita
with h5py.File(general_path + f'/1 - Results_Cemaden/Results_{'AC_ 2019'}.h5', "r+") as f:
    chave = "month_13"  # Nome da chave a ser removida
    if chave in f:
        del f[chave]  # Remove a chave do HDF5
        print(f"Chave '{chave}' removida com sucesso!")
    else:
        print(f"Chave '{chave}' não encontrada no arquivo.")


In [45]:
with pd.HDFStore(general_path + f'/1 - Results_Cemaden/Results_{'AL_ 2017'}.h5') as hdf:
    keys = hdf.keys()  # Obtém todas as chaves do arquivo HDF
    for key in keys:  # Itera por cada chave
        df = pd.read_hdf(hdf, key=key)  # Lê os dados de cada chave
        print(f"Chave: {key}, Número de linhas: {len(df)}")


Chave: /annual, Número de linhas: 21
Chave: /month_01, Número de linhas: 21
Chave: /month_02, Número de linhas: 21
Chave: /month_03, Número de linhas: 21
Chave: /month_04, Número de linhas: 21
Chave: /month_05, Número de linhas: 21
Chave: /month_06, Número de linhas: 21
Chave: /month_07, Número de linhas: 21
Chave: /month_08, Número de linhas: 21
Chave: /month_09, Número de linhas: 21
Chave: /month_10, Número de linhas: 21
Chave: /month_11, Número de linhas: 21
Chave: /month_12, Número de linhas: 21


In [26]:

mit=360
year=2020
state='RS'
name_file= state+ '_'+ str(year)+'.h5'
name_col= state+ '_ '+ str(year)
code='86321000'

In [28]:
start_time = datetime.now()
print(f"Initialize : {state}_{year} {start_time.strftime("%Y-%m-%d %H:%M:%S")}") 
df_rainfall = pd.read_hdf(general_path +'/1 - Organized data/'+ name_file,'table_data')
monthly_results = {m: {"gauge_code": [], "EI30": [], "E_D": [], "rain": []} for m in range(1, 13)}
annual_results = {"gauge_code": [], "EI30": [], "E_D": [], "rain": []}

 
name=code +'_' + str(mit_int)
df_rainfall_events = pd.read_hdf(general_path + '/3 - Rainfall events/'+ name_file, name).drop_duplicates()
df_partial_result = pd.DataFrame(columns=['rain_mm','duration', 'I30', 'RF_KE', 'EI30'])
for evento in range(0,len(df_rainfall_events)):
    beg=df_rainfall_events.iloc[evento,1]
    month = beg.month
    end=df_rainfall_events.iloc[evento,1]+pd.Timedelta(hours=df_rainfall_events.iloc[evento,3])
    df_gauge_rainfall=df_rainfall[df_rainfall['gauge_code']==code]
    df_event = df_gauge_rainfall[(df_gauge_rainfall['datetime'] >= beg) & (df_gauge_rainfall['datetime'] <= end)].reset_index(drop=True)

    if df_event['rain_mm'].sum()>12.7:  #SPECIFICATION AT RUSLE 2
        for i in df_event.index:
            RF=df_event['rain_mm'][i] #mm
            RF_intensity=(RF/10)*60 #mm/h  ###tdel
            #kinectic energy
            KE = calculate_KE(RF_intensity, method) if RF != 0 else 0
            if (KE > 0.283) :KE= 0.283 #max value 
            RF_KE=KE*RF#MJ/ha           
            #calculate I30
            tbeg=df_event.iloc[i,3] 
            tend=df_event.iloc[i,3]+pd.Timedelta(hours=0.5)
            df_temp=df_event[(df_event['datetime'] >= tbeg) & (df_event['datetime'] < tend)]#### < ou <=
            I30=df_temp['rain_mm'].sum()/0.5 #mm/h #intervalo de 0,5 hrs

            df_event.at[i,'Rf_intensity']=RF_intensity
            df_event.at[i,'RF_KE']=RF_KE
            df_event.at[i,'I30']=I30

        #calc ei30       
        EI30=df_event['RF_KE'].sum()*df_event['I30'].max()        
        #set partial results
        df_partial_result.at[evento,'rain_mm']=df_event['rain_mm'].sum() 
        df_partial_result.at[evento,'duration']=df_rainfall_events.iloc[evento,3]        
        df_partial_result.at[evento,'I30']=df_event['I30'].max() 
        df_partial_result.at[evento,'RF_KE']=df_event['RF_KE'].sum()
        df_partial_result.at[evento,'EI30']=EI30
        df_partial_result.at[evento,'month']=month
    
    else:

        df_partial_result.at[evento,'rain_mm']=df_rainfall_events.iloc[evento,2]  
        df_partial_result.at[evento,'duration']=df_rainfall_events.iloc[evento,3]        
        df_partial_result.at[evento,'I30']=0
        df_partial_result.at[evento,'RF_KE']=0
        df_partial_result.at[evento,'EI30']=0
        df_partial_result.at[evento,'month']=month

print(f"  -code: {code} , time: {(datetime.now()-start_time_code).total_seconds():.2f}s")
      

Initialize : RS_2020 2025-04-12 14:23:45


KeyError: 'No object named 86321000_360 in the file'

In [ ]:
df_partial_result

In [104]:
# general path
general_path = 'C:/Users/abner/OneDrive/Documentos/mestrado/DISSERTAÇÃO/scripts'
general_path_tel='C:/Users/abner/OneDrive/Documentos/mestrado/Telemetria-ANA/Scripts e dados'

state = 'MT'

year=2019
mit_int=360

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
#LISTA DE HQ
df_tel_hq=pd.read_excel(general_path+'/Estacoes_telemetria_HQ.xlsx')

In [106]:
start_time = datetime.now()
name_col=state+ '_ '+ str(year)

mit=pd.Timedelta(mit_int, unit ='m')
name_file= state+ '_'+ str(year)+'.h5'

df_info = pd.read_hdf(general_path_tel +'/1 - Organized data/'+ name_file,'table_info')
df_year_hq=df_tel_hq[year].dropna().astype(int)
common_values = set(df_info['gauge_code'].astype(int)).intersection(set(df_year_hq))
hq_name_file=list(map(str, common_values))
df_rainfall = pd.read_hdf(general_path_tel +'/1 - Organized data/'+ name_file,'table_data').dropna(subset=['rain_mm']).reset_index(drop=True)
data = {"gauge_code": [],"EI30_T": [],"E_D": []}


In [110]:
'17389000' in hq_name_file

True

In [172]:
print(f"Começando : {name_col} - {start_time}") 
code = '17389000'


start_time_code = datetime.now()  
name=code +'_' + str(mit_int)
df_rainfall_events = pd.read_hdf(general_path_tel + '/3 - Rainfall events/'+ name_file, name)
df_partial_result = pd.DataFrame(columns=['rain_mm','duration', 'I30', 'RF_KE', 'EI30'])
for evento in range(4,5):
    beg=df_rainfall_events.iloc[evento,1]
    end=df_rainfall_events.iloc[evento,1]+pd.Timedelta(hours=df_rainfall_events.iloc[evento,3])
    df_gauge_rainfall=df_rainfall[df_rainfall['gauge_code']==code]
    df_event = df_gauge_rainfall[(df_gauge_rainfall['datetime'] >= beg) & (df_gauge_rainfall['datetime'] <= end)].reset_index(drop=True)
    for i in df_event.index:
        RF=df_event['rain_mm'][i] #mm
        time_delta=int((df_event.iloc[i,3]-df_event.iloc[i-1,3]).total_seconds() // 60)
        if i==0:RF_intensity=(RF/60)*60 #mm/h  ###tdel
        elif time_delta==0:RF_intensity=(RF/60)*60 #mm/h  ###tdel
        else:RF_intensity=(RF/time_delta)*60 #mm/h  ###tdel
        #kinectic energy
        if RF<=0:KE=0 
        else:
            KE= calculate_KE(RF_intensity, method) if RF != 0 else 0
            
            if (KE > 0.283) :KE= 0.283
            RF_KE=KE*RF#MJ/ha
            tbeg=df_event.iloc[i,3] #df_temp é para os 30 min de análise para o i30
            tend=df_event.iloc[i,3]+pd.Timedelta(hours=0.5)
            df_temp=df_event[(df_event['datetime'] >= tbeg) & (df_event['datetime'] < tend)]#### < ou <=
            I30=df_temp['rain_mm'].sum()/0.5 #mm/h #intervalo de 0,5 hrs

            df_event.at[i,'Rf_intensity']=RF_intensity
            df_event.at[i,'RF_KE']=RF_KE
            df_event.at[i,'I30']=I30

    #calc ei30       
    EI30=df_event['RF_KE'].sum()*df_event['I30'].max()        
    #set partial results
    df_partial_result.at[evento,'rain_mm']=df_event['rain_mm'].sum() 
    df_partial_result.at[evento,'duration']=df_rainfall_events.iloc[evento,3]        
    df_partial_result.at[evento,'I30']=df_event['I30'].max() 
    df_partial_result.at[evento,'RF_KE']=df_event['RF_KE'].sum()
    df_partial_result.at[evento,'EI30']=EI30 

end_time = datetime.now()  # Fim do tempo de iteração
print(f"  -code: {code} , tempo: {(end_time-start_time_code).total_seconds():.2f}s ")

Começando : MT_ 2019 - 2025-04-12 14:45:43.624289
  -code: 17389000 , tempo: 0.19s 


In [170]:
df_partial_result

,rain_mm,duration,I30,RF_KE,EI30
0,8.0,5.0,8.0,1.009483,8.075868
1,2.0,1.0,4.0,0.225565,0.902261
2,34.0,3.0,48.0,7.792143,374.022846
3,4.0,2.0,4.0,0.451131,1.804523
4,496.0,15.0,336.0,137.707197,46269.618337
5,2.0,1.0,4.0,0.225565,0.902261
6,2.0,1.0,4.0,0.225565,0.902261
7,190.0,6.0,348.0,52.60012,18304.841652
8,292.0,6.0,372.0,81.231166,30217.99367
9,22.0,3.0,24.0,4.075762,97.818299


In [176]:
(df_event['RF_KE']).sum()*336.0

46269.61833678155

In [174]:
df_event

,gauge_code,city,state,datetime,rain_mm,Rf_intensity,RF_KE,I30
0,17389000,JUARA,MT,2019-01-04 00:00:00,168.0,168.0,47.544000,336.0
1,17389000,JUARA,MT,2019-01-04 01:00:00,18.0,18.0,4.361017,36.0
2,17389000,JUARA,MT,2019-01-04 02:00:00,18.0,18.0,4.361017,36.0
3,17389000,JUARA,MT,2019-01-04 03:00:00,0.0,NaN,NaN,NaN
4,17389000,JUARA,MT,2019-01-04 04:00:00,48.0,48.0,13.584000,96.0
5,17389000,JUARA,MT,2019-01-04 05:00:00,0.0,NaN,NaN,NaN
6,17389000,JUARA,MT,2019-01-04 06:00:00,0.0,NaN,NaN,NaN
7,17389000,JUARA,MT,2019-01-04 07:00:00,18.0,18.0,4.361017,36.0
8,17389000,JUARA,MT,2019-01-04 08:00:00,26.0,26.0,6.896145,52.0
9,17389000,JUARA,MT,2019-01-04 09:00:00,0.0,NaN,NaN,NaN


In [126]:
df_rainfall_events

,gauge_code,date_hour_begin,rain_mm,duration_hour,intensity_mm_hour,dry_time_hour,rainfall_of_pulses
0,17389000,2019-01-01 03:00:00,8.0,5.0,1.600000,2.0,3.0
1,17389000,2019-01-01 23:00:00,2.0,1.0,2.000000,0.0,1.0
2,17389000,2019-01-02 08:00:00,34.0,3.0,11.333333,0.0,3.0
3,17389000,2019-01-02 23:00:00,4.0,2.0,2.000000,0.0,2.0
4,17389000,2019-01-04 00:00:00,496.0,15.0,33.066667,7.0,8.0
5,17389000,2019-01-05 03:00:00,2.0,1.0,2.000000,0.0,1.0
6,17389000,2019-01-06 14:00:00,2.0,1.0,2.000000,0.0,1.0
7,17389000,2019-01-06 22:00:00,190.0,6.0,31.666667,3.0,3.0
8,17389000,2019-01-07 13:00:00,292.0,6.0,48.666667,0.0,6.0
9,17389000,2019-01-08 06:00:00,22.0,3.0,7.333333,0.0,3.0


In [102]:
df_partial_result['rain_mm'].sum()

1225.4000000000008

In [92]:
pd.read_hdf(general_path_tel + '/3 - Rainfall events/RS_2020.h5', key='86321000_360')['rain_mm'].sum()

1237.6000000000001

In [96]:
pd.read_hdf(general_path + '/Rasults_erosivity/RS_2020.h5', key='86321000_360')['rain_mm'].sum()

FileNotFoundError: File C:/Users/abner/OneDrive/Documentos/mestrado/DISSERTAÇÃO/scripts/Rasults_erosivity/RS_2020.h5 does not exist